In [ ]:
# 📓 05_plot_diameters.ipynb

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from src.mesh_utils import load_mesh
from src.io import load_point_ids

# Load data
mesh = load_mesh('data/template_mesh.vtk')
ids = load_point_ids('output/pa_point_ids.json')
points = mesh.points[ids]

# Center the points
points_centered = points - np.mean(points, axis=0)

# Apply PCA
pca = PCA(n_components=2)
pca.fit(points_centered)
proj = pca.transform(points_centered)

# Get major/minor axes ranges
x_range = np.max(proj[:, 0]) - np.min(proj[:, 0])
y_range = np.max(proj[:, 1]) - np.min(proj[:, 1])

# Plot
plt.figure(figsize=(6, 6))
plt.scatter(proj[:, 0], proj[:, 1], alpha=0.5)
plt.gca().set_aspect('equal')

# Draw major axis
plt.plot([np.min(proj[:, 0]), np.max(proj[:, 0])], [0, 0], 'r--', label=f'Major ~ {x_range:.2f}')
plt.plot([0, 0], [np.min(proj[:, 1]), np.max(proj[:, 1])], 'b--', label=f'Minor ~ {y_range:.2f}')

plt.title('Pulmonary Artery Region - PCA Projection')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 📓 05_plot_diameters.ipynb

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from skimage.measure import EllipseModel
from src.mesh_utils import load_mesh
from src.io import load_point_ids

# Load data
mesh = load_mesh('data/template_mesh.vtk')
ids = load_point_ids('output/pa_point_ids.json')
points = mesh.points[ids]

# Center the points
points_centered = points - np.mean(points, axis=0)

# PCA projection
pca = PCA(n_components=2)
pca.fit(points_centered)
proj = pca.transform(points_centered)

# Fit ellipse to projected points
model = EllipseModel()
model.estimate(proj[:, :2])
xc, yc, a, b, theta = model.params

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(proj[:, 0], proj[:, 1], alpha=0.5, label='Intersection Points')

ellipse = plt.matplotlib.patches.Ellipse((xc, yc), 2*a, 2*b, np.rad2deg(theta),
                                          edgecolor='darkgreen', facecolor='none', lw=2, label='Fitted Ellipse')
ax.add_patch(ellipse)

# Annotate diameters
ax.text(xc, yc - 1.5 * b, f"Major = {2*a:.2f}", color='darkred', fontsize=10, ha='center')
ax.text(xc + 1.5 * a, yc, f"Minor = {2*b:.2f}", color='blue', fontsize=10, ha='center')

ax.set_aspect('equal')
ax.set_title('PA Region - Ellipse Fit with Annotated Diameters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# Method 1: Principal Component Analysis (PCA)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assume points_2d is a numpy array of shape (N, 2)
# These are the 2D points obtained from slicing the mesh

# Center the data
centered = points_2d - np.mean(points_2d, axis=0)

# Apply PCA
pca = PCA(n_components=2)
pca.fit(centered)

# Get the lengths of the principal axes (2 * standard deviation)
major_length = 2 * np.sqrt(pca.explained_variance_[0])
minor_length = 2 * np.sqrt(pca.explained_variance_[1])

# Get the principal directions
major_axis = pca.components_[0] * major_length / 2
minor_axis = pca.components_[1] * minor_length / 2
origin = np.mean(points_2d, axis=0)

# Plot
plt.figure(figsize=(6, 6))
plt.scatter(points_2d[:, 0], points_2d[:, 1], alpha=0.6, label="Intersection Points")

plt.plot(
    [origin[0] - major_axis[0], origin[0] + major_axis[0]],
    [origin[1] - major_axis[1], origin[1] + major_axis[1]],
    color="blue", label="Major Axis"
)
plt.plot(
    [origin[0] - minor_axis[0], origin[0] + minor_axis[0]],
    [origin[1] - minor_axis[1], origin[1] + minor_axis[1]],
    color="red", label="Minor Axis"
)

plt.text(origin[0] + major_axis[0], origin[1] + major_axis[1],
         f"Major: {major_length:.2f} mm", color='blue')
plt.text(origin[0] + minor_axis[0], origin[1] + minor_axis[1],
         f"Minor: {minor_length:.2f} mm", color='red')

plt.title("PCA-based Diameter Estimation")
plt.axis("equal")
plt.legend()
plt.show()


In [ ]:
#Method 2: Ellipse Fitting (Using skimage)
from skimage.measure import EllipseModel

# Fit ellipse to the 2D points
ellipse = EllipseModel()
success = ellipse.estimate(points_2d)

if success:
    xc, yc, a, b, theta = ellipse.params  # Center, axes, rotation

    # Make sure a is the major axis
    major_length = 2 * max(a, b)
    minor_length = 2 * min(a, b)

    # Plot
    t = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = xc + a * np.cos(t) * np.cos(theta) - b * np.sin(t) * np.sin(theta)
    ellipse_y = yc + a * np.cos(t) * np.sin(theta) + b * np.sin(t) * np.cos(theta)

    plt.figure(figsize=(6, 6))
    plt.scatter(points_2d[:, 0], points_2d[:, 1], alpha=0.6, label="Intersection Points")
    plt.plot(ellipse_x, ellipse_y, 'g--', label="Fitted Ellipse")
    plt.text(xc + a, yc, f"Major: {major_length:.2f} mm", color='blue')
    plt.text(xc, yc + b, f"Minor: {minor_length:.2f} mm", color='red')
    plt.title("Ellipse Fit Diameter Estimation")
    plt.axis("equal")
    plt.legend()
    plt.show()
else:
    print("Ellipse fitting failed.")
